# **Finetuned ResNet50**

## **Imports**

In [32]:
import json
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from sklearn.preprocessing import LabelEncoder

from utils import load_images
from utils import TrainingPlot

## **Parameters**

In [2]:
MODEL_VERSION = "_resNet50_1.2"

METADATA = pd.read_csv('../src/meta_data.csv')
IMAGE_FOLDER = "../src/final_data"
IMG_HEIGHT = 128
IMG_WIDTH = 128
NUM_CLASSES = 19
FREEZED_LAYERS = 200
TEST_SPLIT = 0.3
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.0001

## **Load Images & Labels**

In [13]:
METADATA = pd.read_csv('../src/meta_data.csv')

# Load images and labels
image_folder = '../src/final_data'
images, labels = load_images(image_folder, METADATA, IMG_HEIGHT, IMG_WIDTH)

# Normalize images
images = images / 255.0

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Train / Test Split
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

del images
del labels
del labels_categorical

Processing: 100%|██████████| 57141/57141 [01:37<00:00, 583.31it/s]


In [33]:
# create a mapping of all labels
label_mapping = {}

for i in range(len(label_encoder.classes_)):
    label_mapping[i] = label_encoder.inverse_transform([i])[0]

file_path = f'../src/models/resNet50/label_mapping_{MODEL_VERSION}.json'
with open(file_path, 'w') as file:
    json.dump(label_mapping, file, indent=4)
print(label_mapping)

{0: '1 Series', 1: '2 Series', 2: '2 Series Active Tourer', 3: '4 Series Gran Coupe', 4: '5 Series', 5: '911', 6: 'A Class', 7: 'C Class', 8: 'E Class', 9: 'Golf', 10: 'M4', 11: 'Macan', 12: 'Passat', 13: 'Scirocco', 14: 'Touareg', 15: 'X3', 16: 'X5', 17: 'X6', 18: 'up!'}


## **Create Model**

In [8]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = base_model.output
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=HeNormal())(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])


## **Training**

In [9]:
# Train the model
#checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='loss', save_best_only=True, mode='min')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, mode='min')

# Train the model
history = model.fit(X_train, y_train,
    validation_split=TEST_SPLIT,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, TrainingPlot(MODEL_VERSION, "../src/models/resNet50")]
)

Epoch 1/50
500/500 [==============================] - 1798s 4s/step - loss: 2.2830 - accuracy: 0.2925 - val_loss: 2.9246 - val_accuracy: 0.0750
Epoch 2/50
500/500 [==============================] - 1827s 4s/step - loss: 1.0573 - accuracy: 0.6785 - val_loss: 1.1356 - val_accuracy: 0.6778
Epoch 3/50
500/500 [==============================] - 1794s 4s/step - loss: 0.5947 - accuracy: 0.8205 - val_loss: 0.5810 - val_accuracy: 0.8270
Epoch 4/50
500/500 [==============================] - 1786s 4s/step - loss: 0.4098 - accuracy: 0.8749 - val_loss: 0.4934 - val_accuracy: 0.8539
Epoch 5/50
500/500 [==============================] - 1802s 4s/step - loss: 0.3164 - accuracy: 0.9080 - val_loss: 0.6680 - val_accuracy: 0.8119
Epoch 6/50
 30/500 [>.............................] - ETA: 25:16 - loss: 0.2947 - accuracy: 0.9141

KeyboardInterrupt: 

In [10]:
model.save(f"../src/models/resNet50/model_v{MODEL_VERSION}.keras")

In [11]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

 39/215 [====>.........................] - ETA: 2:03 - loss: 0.5642 - accuracy: 0.8377

KeyboardInterrupt: 

In [ ]:
del X_train
del X_test
del y_train
del y_test
del images
del labels
del label_encoder
del labels_encoded
del model